# Explore FVCOM GOM3 Hindcast on AWS Public Data

In [1]:
import xarray as xr
import zarr
import intake
import xugrid as xu
import hvplot.xugrid

In [2]:
intake_catalog_url = 'https://umassd-fvcom.s3.amazonaws.com/gom3/hindcast/umassd-fvcom-gom3.yml'
cat = intake.open_catalog(intake_catalog_url)

In [3]:
list(cat)

['UMASSD-FVCOM-GOM3-zarr2', 'UMASSD-FVCOM-GOM3-zarr3']

In [4]:
if zarr.__version__[0]=='2':
    dataset = 'UMASSD-FVCOM-GOM3-zarr2' 
elif zarr.__version__[0]=='3':
    dataset = 'UMASSD-FVCOM-GOM3-zarr3'

In [5]:
%%time
ds = cat[dataset].to_dask()

CPU times: user 1.34 s, sys: 277 ms, total: 1.61 s
Wall time: 2.98 s


In [6]:
ds

<xarray.Dataset> Size: 45TB
Dimensions:             (time: 341881, node: 48451, four: 4, nele: 90415,
                         three: 3, siglev: 46, maxnode: 11, maxelem: 9,
                         siglay: 45)
Coordinates:
    siglay              (siglay, node) float32 9MB dask.array<chunksize=(45, 48451), meta=np.ndarray>
    siglev              (siglev, node) float32 9MB dask.array<chunksize=(46, 48451), meta=np.ndarray>
  * time                (time) datetime64[ns] 3MB 1978-01-01 ... 2017-01-01
    x                   (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
    y                   (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
Dimensions without coordinates: node, four, nele, three, maxnode, maxelem
Data variables: (12/49)
    ML_Depth            (time, node) float32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    ML_Nsiglay          (time, node) int32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    a1u                 (four, nele) float32 1MB dask.array<chunksize=(4, 90415), meta=np.ndarray>
    a2u                 (four, nele) float32 1MB dask.array<chunksize=(4, 90415), meta=np.ndarray>
    art1                (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
    art2                (node) float32 194kB dask.array<chunksize=(48451,), meta=np.ndarray>
    ...                  ...
    wet_nodes           (time, node) int32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    wet_nodes_prev_int  (time, node) int32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
    ww                  (time, siglay, nele) float32 6TB dask.array<chunksize=(1, 45, 90415), meta=np.ndarray>
    xc                  (nele) float32 362kB dask.array<chunksize=(90415,), meta=np.ndarray>
    yc                  (nele) float32 362kB dask.array<chunksize=(90415,), meta=np.ndarray>
    zeta                (time, node) float32 66GB dask.array<chunksize=(1, 48451), meta=np.ndarray>
Attributes: (12/14)
    title:                       GOM3 Nesting  by Yf.Sun@umassd.edu
    institution:                 School for Marine Science and Technology
    source:                      FVCOM_3.0
    history:                     model started at: 22/12/2011   16:41
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    Conventions:                 CF-1.0
    ...                          ...
    Tidal_Forcing:               TIDAL ELEVATION FORCING IS OFF!
    River_Forcing:               THERE ARE 50 RIVERS IN THIS MODEL.\nRIVER IN...
    GroundWater_Forcing:         GROUND WATER FORCING IS OFF!
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    Surface_Wind_Forcing:        FVCOM variable surface Wind forcing:\nFILE N...
    Surface_PrecipEvap_Forcing:  FVCOM periodic surface precip forcing:\nFILE...

In [7]:
def add_ugrid_metadata(ds):
    """
    Assigns UGRID convention metadata to an xarray Dataset.
    """
    mesh_name = "mesh_topology"

    mesh_attrs = {
        "cf_role": "mesh_topology",
        "topology_dimension": 2,
        "node_coordinates": "lon lat",
        "face_coordinates": "lonc latc",
        "face_node_connectivity": "nv",
        "face_dimension": "nele"
    }

    if mesh_name not in ds:
        ds = ds.assign({mesh_name: xr.DataArray(0, attrs=mesh_attrs)})
    else:
        ds[mesh_name].attrs.update(mesh_attrs)

    if "nv" in ds and ds.nv.dims == ("three", "nele"):
        ds["nv"] = ds["nv"].transpose("nele", "three")

    if "nv" in ds:
        ds.nv.attrs.update({
            "cf_role": "face_node_connectivity",
            "start_index": 1
        })

    for var in ds.data_vars:
        if "node" in ds[var].dims or "nele" in ds[var].dims:
            ds[var].attrs["mesh"] = mesh_name
            ds[var].attrs["location"] = (
                "face" if "nele" in ds[var].dims else "node"
            )

    return ds

In [8]:
ds_ugrid = add_ugrid_metadata(ds)

In [9]:
%%time
uds = xu.UgridDataset(ds_ugrid)

CPU times: user 432 ms, sys: 45.6 ms, total: 477 ms
Wall time: 2.31 s


## Visualize data on the triangular mesh

In [ ]:
%%time
uds['temp'].hvplot.trimesh(
    rasterize=True,
    geo=True,
    tiles='OSM',
    cmap='turbo',
    colorbar=True,
    width=600,
    height=400,
)

In [ ]:
%%time
uds['salinity'].hvplot.trimesh(
    rasterize=True,
    geo=True,
    tiles='OSM',
    cmap='turbo',
    colorbar=True,
    width=600,
    height=400,
)

In [ ]:
%%time
uds['h'].hvplot.trimesh(
    rasterize=True,
    geo=True,
    tiles='OSM',
    cmap='turbo',
    colorbar=True,
    width=600,
    height=400,
)